In [1]:
from tkinter import Tk, Canvas
import random

## Grid size and game globals

In [2]:
WIDTH = 800
HEIGHT = 600
SEG_SIZE = 20
IN_GAME = True
GAME_MODE = 0

## Creating Apple at random locations

In [3]:
def create_apple():
    global APPLE, apple_posx, apple_posy
    apple_posx = SEG_SIZE * random.randint(1, (WIDTH - SEG_SIZE) / SEG_SIZE)
    apple_posy = SEG_SIZE * random.randint(1, (HEIGHT - SEG_SIZE) / SEG_SIZE)
    APPLE = c.create_oval(apple_posx, apple_posy,
                          apple_posx + SEG_SIZE, apple_posy + SEG_SIZE,
                          fill="red")

## Creating apples that are reachable

In [4]:
def create_apple_reachable(period, x1, y1, x2, y2):
    global APPLE, apple_posx, apple_posy
    
    attempts = 0
    while attempts == 0 or (
        abs(apple_posx - x1) % period == 0 and
        abs(apple_posx - x2) % period == 0 
    ):
        apple_posx = SEG_SIZE * random.randint(1, (WIDTH - SEG_SIZE) / SEG_SIZE)
        attempts += 1
        if 1000 < attempts:
            break
        
    attempts = 0
    while attempts == 0 or (
        abs(apple_posy - y1) % period == 0 and
        abs(apple_posy - y2) % period == 0
    ):
        apple_posy = SEG_SIZE * random.randint(1, (HEIGHT - SEG_SIZE) / SEG_SIZE)
        attempts += 1
        if 1000 < attempts:
            break
        
    APPLE = c.create_oval(apple_posx, apple_posy,
                              apple_posx + SEG_SIZE, apple_posy + SEG_SIZE,
                              fill="red")

## Creating segment class which is a snake scale

In [5]:
class Segment(object):
    def __init__(self, x, y, color=None):
        if not color:
            self.instance = c.create_rectangle(x, y, x + SEG_SIZE, y + SEG_SIZE, fill="white")
        else:
            self.instance = c.create_rectangle(x, y, x + SEG_SIZE, y + SEG_SIZE, fill=color)

## Creating Snake class

In [6]:
class Snake(object):
    def __init__(self, segments, color=None):
        self.segments = segments
        self.color = color
        # Possible directions of movement
        self.mapping = {"Down": (0, 1), "Right": (1, 0),
                        "Up": (0, -1), "Left": (-1, 0)}
        # Initial snake movement direction
        self.vector = self.mapping["Right"]

    # Snake movement
    def move(self):
        for index in range(len(self.segments) - 1):
            segment = self.segments[index].instance
            x1, y1, x2, y2 = c.coords(self.segments[index + 1].instance)
            c.coords(segment, x1, y1, x2, y2)

        x1, y1, x2, y2 = c.coords(self.segments[-2].instance)
        c.coords(self.segments[-1].instance,
                 x1 + self.vector[0] * SEG_SIZE, y1 + self.vector[1] * SEG_SIZE,
                 x2 + self.vector[0] * SEG_SIZE, y2 + self.vector[1] * SEG_SIZE)

    # Snake getting fatter: Adding a segment to the snake body
    def add_segment(self, color=None):
        last_seg = c.coords(self.segments[0].instance)
        x = last_seg[2] - SEG_SIZE
        y = last_seg[3] - SEG_SIZE
        self.segments.insert(0, Segment(x, y, color))

    # Change of direction initiated by keyboard
    def change_direction(self, event):
        if event.keysym in self.mapping:
            self.vector = self.mapping[event.keysym]
            
    # Change of direction initiated by AI
    def change_direction_ai(self, direction):
        if direction in self.mapping:
            #print("changing direction to:", direction)
            self.vector = self.mapping[direction]

    def reset_snake(self):
        for segment in self.segments:
            c.delete(segment.instance)

## Function for creating snake

In [7]:
def create_snake(color = None):
    if not color:
        # creating snake segments
        segments = [Segment(SEG_SIZE, SEG_SIZE),
                    Segment(SEG_SIZE*2, SEG_SIZE),
                    Segment(SEG_SIZE*3, SEG_SIZE)]
        return Snake(segments)
    
    else:
        # creating snake segments with color at random locations
        posx = SEG_SIZE * random.randint(1, (WIDTH - SEG_SIZE) / SEG_SIZE)
        posy = SEG_SIZE * random.randint(1, (HEIGHT - SEG_SIZE) / SEG_SIZE)
        segments = [Segment(posx, posy, color),
                    Segment(posx + SEG_SIZE, posy, color),
                    Segment(posx + SEG_SIZE*2, posy, color),
                    Segment(posx + SEG_SIZE*3, posy, color),
                    Segment(posx + SEG_SIZE*4, posy, color)]
        return Snake(segments, color)

## Function for starting game

In [8]:
def start_game():
    global r_s, y_s
    create_apple()
    r_s = create_snake('red')
    y_s = create_snake('yellow')
    main()

## Function for setting state

In [9]:
def set_state(item, state):
    c.itemconfigure(item, state=state)

## Function for starting new game in user has clicked start game

In [10]:
def clicked(event):
    global GAME_MODE
    
    r_s.reset_snake()
    y_s.reset_snake()
    IN_GAME = True
    GAME_MODE = 3
    c.delete(APPLE)
    
    c.itemconfigure(game_over_text, state='hidden')
    c.itemconfigure(remote_simulate_text, state='hidden')
    start_game()

## Setting up window

In [11]:
root = Tk()
root.title("Snake game")

''

In [12]:
c = Canvas(root, width=WIDTH, height=HEIGHT, bg="#003300")
c.grid()

c.focus_set()
game_over_text = c.create_text(WIDTH/2, HEIGHT/2, text="Game over!",
                               font='Arial 20', fill='red',
                               state='hidden')

remote_simulate_text = c.create_text(WIDTH/2, HEIGHT- HEIGHT/5,
                             font='Arial 20',
                             fill='yellow',
                             text="Click here to start game",
                             state='hidden')

c.tag_bind(remote_simulate_text, "<Button-1>", clicked)

'1837923956672clicked'

## Function for sending data to and receiving data from snake server

In [13]:
import socket, keepalive

def feed_snake_server(host, port, game_state):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        keepalive.set(s)
        s.connect((host, port))
        while True:
            s.sendall(game_state)
            data = s.recv(1024)
            print("Client received:", repr(data))
            received = yield repr(data)
            
            # forcing modification of game_state
            if received is not None:
                if received == "Close":
                    # close the connection
                    s.sendall(received)
                    data = s.recv(1024)
                    break
                    
                # change the message
                game_state = received
    
    return repr(data)

## Creating gameloop

In [14]:
import time
import random
import socket

# yellow and red snake generators to call yellow and red snake servers
gen_yellow = None
gen_red = None

def main():
    global GAME_MODE, IN_GAME, APPLE, apple_posx, apple_posy
    global game_heartbeats, snake_immunity_period, game_period, yellow_immunity, red_immunity
    global gen_yellow, gen_red
        
    # Remote mode: dueling snakes with remote game servers AI actions
    if GAME_MODE == 3:
        y_s.move()
        r_s.move()
        yellow_head_coords = c.coords(y_s.segments[-1].instance)
        red_head_coords = c.coords(r_s.segments[-1].instance)
        
        # Yellow snake
        yx1, yy1, yx2, yy2 = yellow_head_coords
        
        # Red snake
        rx1, ry1, rx2, ry2 = red_head_coords
        
        # report initial snake positions
        if game_heartbeats == 0:
            print()
            print("***************** NEW GAME ***************** ")
            print("Started at", time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime()))
            print("yellow snake starts at", yellow_head_coords)
            print("red snake starts at", red_head_coords)
        
        # game info
        game_info = yellow_head_coords + red_head_coords + [apple_posx, apple_posy]
        
        # query snake servers every game_period heartbeats for any change of direction
        if game_heartbeats % game_period == 0:
            # Get command from remote yellow server
            b = str(game_info)[1:-1].encode('utf-8')
            
            start = time.time()
            if game_heartbeats == 0:
                gen_yellow = feed_snake_server(yellow_snake_ip, yellow_snake_port, b)
                direction = next(gen_yellow)[2:-1]
            else:
                direction = gen_yellow.send(b)[2:-1]
            end = time.time()
            
            if direction != "Straight":
                y_s.change_direction_ai(direction)
                print("yellow snake turned", direction)
            if 1 < end - start:
                print("Yellow snake server took", end - start, "seconds to reply!")

            # Get command from remote red server
            start = time.time()
            if game_heartbeats == 0:
                gen_red = feed_snake_server(red_snake_ip, red_snake_port, b)
                direction = next(gen_red)[2:-1]
            else:
                direction = gen_red.send(b)[2:-1]
            end = time.time()
            
            if direction != "Straight":
                r_s.change_direction_ai(direction)
                print("red snake turned", direction)
            if 1 < end - start:
                print("Red snake server took", end - start, "seconds to reply!")
            
        # eating apples, definitely get fatter!
        # If both snakes reach apple at the same time, yellow snake wins
        if yellow_head_coords == c.coords(APPLE):
            y_s.add_segment("yellow")
            yellow_immunity += snake_immunity_period + 1
            print("yellow snake ate apple at", APPLE)
            c.delete(APPLE)
            #create_apple()
            create_apple_reachable(game_period, red_head_coords[0], red_head_coords[1],
                                             yellow_head_coords[0], yellow_head_coords[1])
            print("new apple at", APPLE)
        elif red_head_coords == c.coords(APPLE):
            r_s.add_segment("red")
            red_immunity += snake_immunity_period + 1
            print("red snake ate apple at", APPLE)
            c.delete(APPLE)
            #create_apple()
            create_apple_reachable(game_period, red_head_coords[0], red_head_coords[1],
                                             yellow_head_coords[0], yellow_head_coords[1])
            print("new apple at", APPLE)

        # Yellow snake kills red snake
        if 0 == red_immunity:
            for index in range(len(y_s.segments) - 1):
                if red_head_coords == c.coords(y_s.segments[index].instance):
                    print("Yellow snake killed red snake after", game_heartbeats, "game heartbeats!")
                    IN_GAME = False
                    GAME_MODE = 0

        # Red snake kills yellow snake
        if 0 == yellow_immunity:
            for index in range(len(r_s.segments) - 1):
                if yellow_head_coords == c.coords(r_s.segments[index].instance):
                    print("Red snake killed yellow snake after", game_heartbeats, "game heartbeats!")
                    IN_GAME = False
                    GAME_MODE = 0
        
        # game state variables
        if 0 < yellow_immunity:
            yellow_immunity -= 1
        if 0 < red_immunity:
            red_immunity -= 1
        game_heartbeats += 1
        root.after(50, main)   

    # if crash, end game
    else:
        set_state(game_over_text, 'normal')
        set_state(remote_simulate_text, 'normal')
        game_heartbeats = 0
        yellow_immunity = 0
        red_immunity = 0

## Configuring game settings

In [15]:
game_heartbeats = 0
snake_immunity_period, game_period = 2, 1
yellow_immunity, red_immunity = 0, 0
yellow_snake_ip, red_snake_ip = "127.0.0.1", "127.0.0.1"
yellow_snake_port, red_snake_port = 5001, 5002

## Starting game

In [16]:
start_game()
root.mainloop()


***************** NEW GAME ***************** 
Started at 2025-04-25 22:17:44
yellow snake starts at [780.0, 180.0, 800.0, 200.0]
red snake starts at [240.0, 580.0, 260.0, 600.0]
Client received: b'Left'
yellow snake turned Left
Client received: b'Up'
red snake turned Up
Client received: b'Left'
yellow snake turned Left
Client received: b'Right'
red snake turned Right
Client received: b'Down'
yellow snake turned Down
Client received: b'Right'
red snake turned Right
Client received: b'Left'
yellow snake turned Left
Client received: b'Right'
red snake turned Right
Client received: b'Down'
yellow snake turned Down
Client received: b'Right'
red snake turned Right
Client received: b'Left'
yellow snake turned Left
Client received: b'Right'
red snake turned Right
Client received: b'Down'
yellow snake turned Down
Client received: b'Right'
red snake turned Right
Client received: b'Left'
yellow snake turned Left
Client received: b'Up'
red snake turned Up
Client received: b'Down'
yellow snake tur

Client received: b'Left'
yellow snake turned Left
Client received: b'Down'
red snake turned Down
Client received: b'Left'
yellow snake turned Left
Client received: b'Left'
red snake turned Left
Client received: b'Down'
yellow snake turned Down
Client received: b'Down'
red snake turned Down
Client received: b'Left'
yellow snake turned Left
Client received: b'Left'
red snake turned Left
Client received: b'Down'
yellow snake turned Down
Client received: b'Down'
red snake turned Down
Client received: b'Down'
yellow snake turned Down
Client received: b'Right'
red snake turned Right
red snake ate apple at 36
new apple at 38
Client received: b'Right'
yellow snake turned Right
Client received: b'Right'
red snake turned Right
Client received: b'Right'
yellow snake turned Right
Client received: b'Right'
red snake turned Right
Client received: b'Right'
yellow snake turned Right
Client received: b'Right'
red snake turned Right
Client received: b'Right'
yellow snake turned Right
Client received: b'

Client received: b'Right'
yellow snake turned Right
Client received: b'Down'
red snake turned Down
red snake ate apple at 42
new apple at 44
Client received: b'Right'
yellow snake turned Right
Client received: b'Right'
red snake turned Right
Client received: b'Right'
yellow snake turned Right
Client received: b'Up'
red snake turned Up
Client received: b'Right'
yellow snake turned Right
Client received: b'Right'
red snake turned Right
Client received: b'Up'
yellow snake turned Up
Client received: b'Up'
red snake turned Up
Client received: b'Up'
yellow snake turned Up
Client received: b'Right'
red snake turned Right
Client received: b'Up'
yellow snake turned Up
Client received: b'Up'
red snake turned Up
Client received: b'Up'
yellow snake turned Up
Client received: b'Right'
red snake turned Right
Client received: b'Up'
yellow snake turned Up
Client received: b'Up'
red snake turned Up
Client received: b'Up'
yellow snake turned Up
Client received: b'Right'
red snake turned Right
Client rec